# Proxy Rotation (Note: Use EliteProxies)

In [27]:
import requests
from bs4 import BeautifulSoup

In [3]:
res = requests.get('https://free-proxy-list.net')
content = BeautifulSoup(res.text, 'lxml')
table = content.find('table')
rows = table.find_all('tr')
cols = [[col.text for col in row.find_all('td')] for row in rows]

proxies = []
proxy_index = 0

for col in cols:
    try:
        if col[4] == 'elite proxy' and col[6] == 'yes':
            proxies.append('https://' + col[0] + ':' + col[1])
    except:
        pass
# print (proxies)

def fetch(url, params):
    global proxy_index
    
    while proxy_index < len(proxies):
        try:
            print('Trying proxy:', proxies[proxy_index])
            res = requests.get(url, proxies={'https': proxies[proxy_index]}, params=params, timeout=5)
            return res
            
        except:
            print('Bad proxy...')
            proxy_index += 1

for page in range(0, 4):
    params = {'page': page}
    res = fetch('https://www.google.com/', params=params)
    proxy_index += 1
    print('ip', res.json()['ip'])
    print('url:', res.json()['url'])  

Trying proxy: https://47.240.160.90:10001
Bad proxy...
Trying proxy: https://195.158.30.232:3128
Bad proxy...
Trying proxy: https://5.161.105.105:80
Bad proxy...
Trying proxy: https://43.225.23.132:80
Bad proxy...
Trying proxy: https://150.136.108.121:3128
Bad proxy...
Trying proxy: https://85.25.91.141:15333
Bad proxy...
Trying proxy: https://85.195.104.71:80
Bad proxy...
Trying proxy: https://38.108.119.176:59394
Bad proxy...
Trying proxy: https://50.231.95.3:8080
Bad proxy...
Trying proxy: https://18.230.58.67:3128
Bad proxy...
Trying proxy: https://78.46.123.202:80
Bad proxy...
Trying proxy: https://5.189.184.6:80
Bad proxy...
Trying proxy: https://1.10.141.220:54620
Bad proxy...
Trying proxy: https://154.66.210.1:8080
Bad proxy...
Trying proxy: https://151.106.17.122:1080
Bad proxy...
Trying proxy: https://151.106.17.125:1080
Bad proxy...
Trying proxy: https://151.106.17.123:1080
Bad proxy...


AttributeError: 'NoneType' object has no attribute 'json'

# Word Combinations (of 20 Words Given)

In [1]:
words2Scrap=["Table", "Fog", "Wikipedia", "Empire", "Ruin", "Era", "English", "Library", "Twin", "Tower", "Book", "Art",
             "Science", "Poor", "Rich", "Dad", "Heart", "Power", "Medicine", "Borrow" ]

In [2]:
from itertools import combinations

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]

In [3]:
lister= list(combs(words2Scrap))

In [4]:
len(lister)

1048576

In [5]:
len(lister[0])

0

In [6]:
def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://openlibrary.org/search?q=paper+trains&mode=everything"
    return urlMain


In [11]:
# def getSearchTerm4url(typle,index):
#     searchTerm=""
#     for i,d in enumerate(list(typle[index])):
#         if (i!=0):
#             searchTerm+="+"+str(d)
#         else:
#             searchTerm=d
#     return searchTerm

In [7]:
import requests
from bs4 import BeautifulSoup

def getLinksOfPageURL(lister, index,urls):
    if (type(lister) is not str):
        URL =formatSearchTerm4url(lister,index)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        job_elements = soup.find(class_="pagination")

        page_no=0

        if (lister[index]!=()):
            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\n\n\n\n\n\nURL of Index {index}...")
#             try:
            for indexPage,data in enumerate(pages):
                if (indexPage==0):
                    url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+"1"
                    page_no=1
                    print ("Found URL:", (url))
                    urls.append(url)
                else:
                    try:
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        page_no=int(data.text)
                        urls.append(url)
                        print(len(urls))
                    except:
                        pass
                    
                    
#             except:
#                 print("except",len(urls))
#                 return urls

            if (page_no!=0):
                url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+str(page_no)
                getLinksOfPageURL(url,page_no,urls)

        else:
            print(f"NOTE: Lister's Index of {index} has ZERO elements!")
            return []
##############
    
    else:
        
        page = requests.get(lister)
        
        print("Total Pages So Far: ", index)
        page_no=index

        try:
            page = requests.get(lister)
            soup = BeautifulSoup(page.content, "html.parser")
            job_elements = soup.find(class_="pagination")

            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\nFrom Page {index}...")
            for indexPage,data in enumerate(pages):
                    if ("Next" in data.text or "First" in data.text or "Previous" in data.text):
                        pass
                    elif (int(data.text)>page_no):
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        urls.append(url)
                        print(len(urls))
                        page_no=int(data.text)

            getLinksOfPageURL(url,page_no,urls)
        except:
            print("End Of the Line.... No More Next Page")
            print(urls)
            return urls
        return urls

In [45]:
formatSearchTerm4url(lister,0)

'https://openlibrary.org/search?q=paper+trains&mode=everything'

In [8]:
urlFile=[]
for i in range(0,len(lister)):
    urlFile.append(getLinksOfPageURL(lister,i,urlFile))
    print("TOTAL LENGTH OF URLS COLLECTED:::",len(urlFile))
    if (i==1): break
        
#NOTE: total 2865 pages

NOTE: Lister's Index of 0 has ZERO elements!
TOTAL LENGTH OF URLS COLLECTED::: 1






URL of Index 1...
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=1
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=3
3
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=4
4
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=5
5
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=6
6
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=7
7
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=8
8
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=9
9
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=10
10
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=2
Total Pages So Far:  10

From Page 10...
Found URL: https://openlibrary.org/s

In [48]:
import numpy
a = numpy.asarray(urlFile)
numpy.savetxt(f"URLScrapedKeyWords_WITH_{len(urlFile)}_links.csv", a, delimiter=",", fmt='%s')

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# def getLinksOfPageURL(lister, index):
#     urls=[]
    
#     #if Lister array of Index is not empty
#     if (len(lister[index])!=0):
#         URL =formatSearchTerm4url(lister,index)
#         page = requests.get(URL)

#         soup = BeautifulSoup(page.content, "html.parser")
#         job_elements = soup.find(class_="pagination")

#         try:
#             for indexPage,data in enumerate(pages):
#                     print ("Possible URLs found...)
#                     break
            
#             count=1
#             url= f"https://openlibrary.org/search?mode=everythin12&q={getSearchTerm4url(lister,index)}&page="+count
#             if (len(lister[index])!=0):
#                 pages= job_elements.find_all(class_="ChoosePage",href=True)

#                 print(f"\n\n\nURL of Index {index}...")
#                 url=""

                

#                 url="https://openlibrary.org"+data['href']
#                 print ("Found URL:",url )
#                 urls.append(url)
#                 page_no=data.text

#                 page_no=str(int(page_no)+1)
#                 url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+(page_no)
#     #                 try:
#                 urls+=getLinksOfPageURL(url,int(page_no))
           
#         except:
#             print("No other Pages for URL:",formatSearchTerm4url(lister,index))
#             urls.append(formatSearchTerm4url(lister,index))
#             return urls
        
        
#      else:
#         print(f"NOTE: Lister's Index of {index} has ZERO elements!")